# Result Analysis Notebook
This notebooks is used to backtest the processed data and create a performance report

## Key Activates:
    1. Downlaod processed data from data/processed
    2. Create and signals from signals.py
    3. Run a Backtest from backtest.py
    4. Generate a report from attribution.py and future talks about adding to the projects

In [ ]:
# Import required libraries
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import warnings
from datetime import datetime, timedelta
warnings.filterwarnings('ignore')

# Add src directory to path to import custom modules
sys.path.append('../src')
from data_pipeline import download_raw_data, preprocess_data, save_data
from signals import zscore_normalize, order_book_imbalance, etf_constituent_dislocation
from backtest import (walk_forward_split, factor_neutralize, volatility_targeting, 
                     run_backtest, evaluate_performance, plot_performance)
from attribution import (calculate_factor_exposures, attribute_pnl, 
                        generate_attribution_report, plot_attribution)

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries imported successfully!")

In [ ]:
# Load configuration from settings.yaml
with open('../configs/settings.yaml', 'r') as file:
    config = yaml.safe_load(file)

print("Configuration loaded:")
print(f"Tickers: {config['data']['tickers']}")
print(f"Backtest period: {config['backtest']['start_date']} to {config['backtest']['end_date']}")
print(f"Strategy type: {config['strategy']['type']}")
print(f"Initial capital: ${config['backtest']['initial_capital']:,}")

# Extract configuration parameters
tickers = config['data']['tickers']
backtest_start = config['backtest']['start_date']
backtest_end = config['backtest']['end_date']
initial_capital = config['backtest']['initial_capital']
commission = config['backtest']['commission']
strategy_config = config['strategy']
data_paths = config['paths']

In [ ]:
# Create sample processed data (since processed data may not exist yet)
print("Creating sample processed data for backtesting...")

# Generate synthetic but realistic financial data
np.random.seed(42)
start_date = pd.to_datetime(backtest_start)
end_date = pd.to_datetime(backtest_end)
dates = pd.date_range(start=start_date, end=end_date, freq='D')

# Remove weekends to simulate trading days only
trading_days = dates[dates.weekday < 5]

print(f"Creating data for {len(trading_days)} trading days")

# Create sample processed data for each ticker
processed_data = {}
for ticker in tickers:
    n_days = len(trading_days)
    
    # Generate realistic price data with drift and volatility
    initial_price = np.random.uniform(50, 500)  # Random initial price
    daily_returns = np.random.normal(0.0005, 0.02, n_days)  # 0.05% mean, 2% std daily
    
    # Add some autocorrelation to make returns more realistic
    for i in range(1, len(daily_returns)):
        daily_returns[i] += 0.1 * daily_returns[i-1]
    
    # Create cumulative prices
    price_series = initial_price * np.exp(np.cumsum(daily_returns))
    
    # Create OHLCV data
    data = pd.DataFrame(index=trading_days)
    data['Close'] = price_series
    data['Open'] = data['Close'].shift(1) * (1 + np.random.normal(0, 0.005, n_days))
    data['High'] = np.maximum(data['Open'], data['Close']) * (1 + np.abs(np.random.normal(0, 0.01, n_days)))
    data['Low'] = np.minimum(data['Open'], data['Close']) * (1 - np.abs(np.random.normal(0, 0.01, n_days)))
    data['Volume'] = np.random.lognormal(15, 1, n_days).astype(int)  # Log-normal volume
    
    # Calculate returns
    data['Returns'] = data['Close'].pct_change()
    
    # Add technical indicators
    data['SMA_20'] = data['Close'].rolling(20).mean()
    data['SMA_50'] = data['Close'].rolling(50).mean()
    data['RSI'] = calculate_rsi(data['Close'], 14)
    data['MACD'] = calculate_macd(data['Close'])
    data['BB_Upper'], data['BB_Lower'] = calculate_bollinger_bands(data['Close'])
    
    # Add factor exposures (market factors)
    data['Market_Factor'] = np.random.normal(0, 1, n_days)  # Market beta exposure
    data['Size_Factor'] = np.random.normal(0, 0.5, n_days)  # Size factor exposure
    data['Value_Factor'] = np.random.normal(0, 0.3, n_days)  # Value factor exposure
    
    # Clean data
    data = data.dropna()
    processed_data[ticker] = data

def calculate_rsi(prices, window=14):
    """Calculate RSI"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def calculate_macd(prices, fast=12, slow=26, signal=9):
    """Calculate MACD"""
    exp1 = prices.ewm(span=fast).mean()
    exp2 = prices.ewm(span=slow).mean()
    return exp1 - exp2

def calculate_bollinger_bands(prices, window=20, std_dev=2):
    """Calculate Bollinger Bands"""
    sma = prices.rolling(window).mean()
    std = prices.rolling(window).std()
    return sma + (std * std_dev), sma - (std * std_dev)

print(f"Created processed data for {len(processed_data)} tickers")
for ticker, data in processed_data.items():
    print(f"{ticker}: {len(data)} observations from {data.index[0].date()} to {data.index[-1].date()}")

In [ ]:
# Step 1: Signal Generation using signals.py
print("Generating trading signals...")

# Combine all data for signal generation
combined_returns = pd.DataFrame()
combined_prices = pd.DataFrame()

for ticker, data in processed_data.items():
    combined_returns[ticker] = data['Returns']
    combined_prices[ticker] = data['Close']

# Generate mean reversion signals using z-score normalization
lookback_period = strategy_config['lookback_period']
entry_threshold = strategy_config['entry_threshold']
exit_threshold = strategy_config['exit_threshold']

signals = pd.DataFrame(index=combined_returns.index)

print(f"Using lookback period: {lookback_period} days")
print(f"Entry threshold: {entry_threshold} standard deviations")
print(f"Exit threshold: {exit_threshold} standard deviations")

for ticker in tickers:
    if ticker in combined_prices.columns:
        prices = combined_prices[ticker]
        
        # Calculate rolling statistics
        rolling_mean = prices.rolling(window=lookback_period).mean()
        rolling_std = prices.rolling(window=lookback_period).std()
        z_score = (prices - rolling_mean) / rolling_std
        
        # Apply z-score normalization using custom function
        try:
            normalized_z = zscore_normalize(z_score.dropna())
            # Realign with original index
            z_score_normalized = pd.Series(index=z_score.index, dtype=float)
            z_score_normalized[normalized_z.index] = normalized_z
        except:
            z_score_normalized = z_score
        
        # Generate signals
        signal = pd.Series(index=prices.index, dtype=float)
        signal[:] = 0  # Default to no position
        
        # Long signal when price is below lower threshold (oversold)
        signal[z_score < -entry_threshold] = 1
        # Short signal when price is above upper threshold (overbought)  
        signal[z_score > entry_threshold] = -1
        # Exit when z-score returns to normal range
        signal[abs(z_score) < exit_threshold] = 0
        
        # Apply signal persistence (don't flip immediately)
        signal = signal.fillna(method='ffill').fillna(0)
        
        signals[f'{ticker}_signal'] = signal
        signals[f'{ticker}_zscore'] = z_score

print(f"Generated signals for {len([c for c in signals.columns if 'signal' in c])} assets")

# Signal quality analysis
signal_stats = {}
for ticker in tickers:
    signal_col = f'{ticker}_signal'
    if signal_col in signals.columns:
        sig = signals[signal_col]
        signal_stats[ticker] = {
            'Long_signals': (sig == 1).sum(),
            'Short_signals': (sig == -1).sum(),
            'No_position': (sig == 0).sum(),
            'Signal_frequency': (sig != 0).sum() / len(sig)
        }

signal_summary = pd.DataFrame(signal_stats).T
print("\nSignal Summary:")
print(signal_summary)

In [ ]:
# Step 2: Signal Visualization
print("Visualizing trading signals...")

# Create signal visualization for top 4 assets
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
axes = axes.flatten()

for i, ticker in enumerate(tickers[:4]):
    ax = axes[i]
    
    # Plot price and z-score
    price = combined_prices[ticker]
    zscore = signals[f'{ticker}_zscore']
    signal = signals[f'{ticker}_signal']
    
    # Create twin axis for z-score
    ax2 = ax.twinx()
    
    # Plot price
    ax.plot(price.index, price, 'b-', label='Price', linewidth=2)
    ax.set_ylabel('Price ($)', color='b')
    ax.tick_params(axis='y', labelcolor='b')
    
    # Plot z-score
    ax2.plot(zscore.index, zscore, 'r-', alpha=0.7, label='Z-Score')
    ax2.axhline(y=entry_threshold, color='orange', linestyle='--', alpha=0.7, label=f'Entry Threshold (±{entry_threshold})')
    ax2.axhline(y=-entry_threshold, color='orange', linestyle='--', alpha=0.7)
    ax2.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
    ax2.set_ylabel('Z-Score', color='r')
    ax2.tick_params(axis='y', labelcolor='r')
    
    # Highlight signal periods
    long_signals = signal == 1
    short_signals = signal == -1
    
    if long_signals.any():
        ax.scatter(price.index[long_signals], price[long_signals], 
                  color='green', marker='^', s=50, alpha=0.7, label='Long Signal')
    if short_signals.any():
        ax.scatter(price.index[short_signals], price[short_signals], 
                  color='red', marker='v', s=50, alpha=0.7, label='Short Signal')
    
    ax.set_title(f'{ticker} - Price and Trading Signals', fontweight='bold')
    ax.legend(loc='upper left')
    ax2.legend(loc='upper right')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Signal distribution analysis
print("\nSignal Distribution Analysis:")
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Z-score distribution
ax1 = axes[0]
for ticker in tickers[:4]:
    zscore = signals[f'{ticker}_zscore'].dropna()
    ax1.hist(zscore, bins=30, alpha=0.6, label=ticker)
ax1.axvline(x=entry_threshold, color='red', linestyle='--', label=f'Entry Threshold (±{entry_threshold})')
ax1.axvline(x=-entry_threshold, color='red', linestyle='--')
ax1.set_xlabel('Z-Score')
ax1.set_ylabel('Frequency')
ax1.set_title('Z-Score Distribution', fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Signal frequency over time
ax2 = axes[1]
monthly_signals = signals[[c for c in signals.columns if 'signal' in c]].resample('M').apply(lambda x: (x != 0).sum())
monthly_signals.plot(kind='bar', ax=ax2, alpha=0.7)
ax2.set_title('Monthly Signal Frequency', fontweight='bold')
ax2.set_xlabel('Month')
ax2.set_ylabel('Number of Signals')
ax2.tick_params(axis='x', rotation=45)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Step 3: Portfolio Construction and Walk-Forward Analysis
print("Setting up walk-forward backtesting framework...")

# Prepare data for backtesting using backtest.py functions
# Combine all data into a single DataFrame for walk-forward analysis
backtest_data = pd.DataFrame(index=combined_returns.index)

# Add returns for all assets
for ticker in tickers:
    if ticker in combined_returns.columns:
        backtest_data[f'{ticker}_return'] = combined_returns[ticker]
        backtest_data[f'{ticker}_signal'] = signals[f'{ticker}_signal']

# Add market factors for factor neutralization
backtest_data['Market_Factor'] = np.random.normal(0, 1, len(backtest_data))
backtest_data['Size_Factor'] = np.random.normal(0, 0.5, len(backtest_data))

# Create portfolio returns based on signals
print("Constructing portfolio based on signals...")

# Equal weight approach with signal-based allocation
n_assets = len(tickers)
max_positions = strategy_config['max_positions']

portfolio_returns = pd.Series(index=backtest_data.index, dtype=float)
portfolio_positions = pd.DataFrame(index=backtest_data.index, columns=tickers)

for date in backtest_data.index[1:]:  # Start from second day
    active_signals = {}
    
    # Get current signals for all assets
    for ticker in tickers:
        signal_col = f'{ticker}_signal'
        if signal_col in backtest_data.columns:
            signal = backtest_data.loc[date, signal_col]
            if signal != 0:  # Non-zero signal
                active_signals[ticker] = signal
    
    # Limit to max positions
    if len(active_signals) > max_positions:
        # Select strongest signals (furthest from zero)
        sorted_signals = sorted(active_signals.items(), key=lambda x: abs(x[1]), reverse=True)
        active_signals = dict(sorted_signals[:max_positions])
    
    # Calculate equal weights for active positions
    if active_signals:
        total_weight = sum(abs(signal) for signal in active_signals.values())
        normalized_weights = {ticker: signal/total_weight for ticker, signal in active_signals.items()}
        
        # Calculate portfolio return for this period
        period_return = 0
        for ticker, weight in normalized_weights.items():
            return_col = f'{ticker}_return'
            if return_col in backtest_data.columns:
                asset_return = backtest_data.loc[date, return_col]
                if not pd.isna(asset_return):
                    period_return += weight * asset_return
                    portfolio_positions.loc[date, ticker] = weight
        
        portfolio_returns.loc[date] = period_return
    else:
        portfolio_returns.loc[date] = 0  # No positions

# Clean and fill missing values
portfolio_returns = portfolio_returns.fillna(0)
portfolio_positions = portfolio_positions.fillna(0)

print(f"Portfolio construction complete. Average daily return: {portfolio_returns.mean():.4f}")
print(f"Portfolio volatility (daily): {portfolio_returns.std():.4f}")
print(f"Non-zero position days: {(portfolio_returns != 0).sum()} out of {len(portfolio_returns)}")

# Apply risk controls using backtest.py functions
print("\nApplying risk controls...")

# Create DataFrame for risk control functions
risk_control_data = pd.DataFrame({
    'Returns': portfolio_returns,
    'Market_Factor': backtest_data['Market_Factor'],
    'Size_Factor': backtest_data['Size_Factor']
})

# Apply factor neutralization
try:
    neutralized_data = factor_neutralize(risk_control_data, factors=['Market_Factor', 'Size_Factor'])
    print("✓ Factor neutralization applied")
except Exception as e:
    print(f"⚠ Factor neutralization failed: {e}")
    neutralized_data = risk_control_data.copy()
    neutralized_data['Neutralized Returns'] = neutralized_data['Returns']

# Apply volatility targeting
target_vol = 0.15  # 15% annualized target volatility
try:
    vol_targeted_data = volatility_targeting(neutralized_data, target_volatility=target_vol)
    print(f"✓ Volatility targeting applied (target: {target_vol:.1%})")
except Exception as e:
    print(f"⚠ Volatility targeting failed: {e}")
    vol_targeted_data = neutralized_data.copy()
    vol_targeted_data['Volatility Targeted Returns'] = vol_targeted_data['Neutralized Returns']

final_returns = vol_targeted_data['Volatility Targeted Returns'].fillna(0)

In [ ]:
# Step 4: Backtest Execution and Performance Analysis
print("Running backtest and performance analysis...")

# Run the backtest using backtest.py functions
backtest_df = pd.DataFrame({
    'Returns': portfolio_returns,
    'Neutralized Returns': neutralized_data['Neutralized Returns'],
    'Volatility Targeted Returns': final_returns
})

# Execute backtest
portfolio_backtest = run_backtest(backtest_df, initial_capital=initial_capital)

# Calculate performance metrics
print("Calculating performance metrics...")
performance_metrics = evaluate_performance(portfolio_backtest)

print("\n" + "="*60)
print("BACKTEST PERFORMANCE RESULTS")
print("="*60)

for metric, value in performance_metrics.items():
    if isinstance(value, float):
        if 'Return' in metric:
            print(f"{metric}: {value:.2%}")
        elif 'Drawdown' in metric:
            print(f"{metric}: {value:.2%}")
        else:
            print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}: {value}")

# Additional performance metrics
returns_series = final_returns
sharpe_ratio = (returns_series.mean() * 252) / (returns_series.std() * np.sqrt(252))
sortino_ratio = (returns_series.mean() * 252) / (returns_series[returns_series < 0].std() * np.sqrt(252))
calmar_ratio = (returns_series.mean() * 252) / abs(performance_metrics['Max Drawdown'])

# Win rate analysis
positive_returns = returns_series[returns_series > 0]
negative_returns = returns_series[returns_series < 0]
win_rate = len(positive_returns) / len(returns_series[returns_series != 0]) if len(returns_series[returns_series != 0]) > 0 else 0

print(f"\nAdditional Metrics:")
print(f"Sharpe Ratio: {sharpe_ratio:.3f}")
print(f"Sortino Ratio: {sortino_ratio:.3f}")
print(f"Calmar Ratio: {calmar_ratio:.3f}")
print(f"Win Rate: {win_rate:.1%}")
print(f"Average Win: {positive_returns.mean():.4f}")
print(f"Average Loss: {negative_returns.mean():.4f}")
print(f"Profit Factor: {positive_returns.sum() / abs(negative_returns.sum()):.2f}")

# Monthly and yearly performance breakdown
monthly_returns = returns_series.resample('M').apply(lambda x: (1 + x).prod() - 1)
yearly_returns = returns_series.resample('Y').apply(lambda x: (1 + x).prod() - 1)

print(f"\nMonthly Statistics:")
print(f"Best Month: {monthly_returns.max():.2%}")
print(f"Worst Month: {monthly_returns.min():.2%}")
print(f"Positive Months: {(monthly_returns > 0).sum()}/{len(monthly_returns)}")

if len(yearly_returns) > 1:
    print(f"\nYearly Returns:")
    for year, ret in yearly_returns.items():
        print(f"{year.year}: {ret:.2%}")

In [ ]:
# Step 5: Performance Visualization
print("Creating performance visualizations...")

# Create comprehensive performance charts
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Chart 1: Cumulative Returns
ax1 = axes[0, 0]
cumulative_returns = (1 + returns_series).cumprod()
portfolio_value = initial_capital * cumulative_returns

ax1.plot(portfolio_value.index, portfolio_value, linewidth=2, label='Portfolio Value')
ax1.axhline(y=initial_capital, color='gray', linestyle='--', alpha=0.7, label='Initial Capital')
ax1.set_title('Portfolio Performance Over Time', fontweight='bold')
ax1.set_ylabel('Portfolio Value ($)')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

# Chart 2: Drawdown
ax2 = axes[0, 1]
rolling_max = cumulative_returns.expanding().max()
drawdown = (cumulative_returns - rolling_max) / rolling_max
ax2.fill_between(drawdown.index, drawdown, 0, alpha=0.7, color='red')
ax2.plot(drawdown.index, drawdown, color='darkred', linewidth=1)
ax2.set_title('Portfolio Drawdown', fontweight='bold')
ax2.set_ylabel('Drawdown (%)')
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.1%}'))
ax2.grid(True, alpha=0.3)

# Chart 3: Rolling Sharpe Ratio
ax3 = axes[1, 0]
rolling_sharpe = returns_series.rolling(252).apply(
    lambda x: (x.mean() * 252) / (x.std() * np.sqrt(252)) if x.std() > 0 else 0
)
ax3.plot(rolling_sharpe.index, rolling_sharpe, linewidth=2)
ax3.axhline(y=1, color='green', linestyle='--', alpha=0.7, label='Sharpe = 1.0')
ax3.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
ax3.set_title('Rolling 1-Year Sharpe Ratio', fontweight='bold')
ax3.set_ylabel('Sharpe Ratio')
ax3.legend()
ax3.grid(True, alpha=0.3)

# Chart 4: Monthly Returns Heatmap
ax4 = axes[1, 1]
monthly_ret_pivot = monthly_returns.to_frame('Returns')
monthly_ret_pivot['Year'] = monthly_ret_pivot.index.year
monthly_ret_pivot['Month'] = monthly_ret_pivot.index.month
monthly_heatmap = monthly_ret_pivot.pivot(index='Year', columns='Month', values='Returns')

sns.heatmap(monthly_heatmap, annot=True, fmt='.1%', cmap='RdYlGn', center=0, 
           ax=ax4, cbar_kws={'label': 'Monthly Return'})
ax4.set_title('Monthly Returns Heatmap', fontweight='bold')
ax4.set_xlabel('Month')
ax4.set_ylabel('Year')

plt.tight_layout()
plt.show()

# Performance comparison chart
fig, ax = plt.subplots(1, 1, figsize=(12, 8))

# Compare different return series
raw_cumulative = (1 + portfolio_returns).cumprod() * initial_capital
neutralized_cumulative = (1 + neutralized_data['Neutralized Returns']).cumprod() * initial_capital
final_cumulative = (1 + final_returns).cumprod() * initial_capital

ax.plot(raw_cumulative.index, raw_cumulative, label='Raw Strategy', alpha=0.8)
ax.plot(neutralized_cumulative.index, neutralized_cumulative, label='Factor Neutralized', alpha=0.8)
ax.plot(final_cumulative.index, final_cumulative, label='Vol Targeted (Final)', alpha=0.8, linewidth=2)

# Add benchmark (buy and hold equal weight)
benchmark_returns = combined_returns.mean(axis=1)
benchmark_cumulative = (1 + benchmark_returns).cumprod() * initial_capital
ax.plot(benchmark_cumulative.index, benchmark_cumulative, label='Equal Weight Benchmark', 
        color='gray', alpha=0.7, linestyle='--')

ax.set_title('Strategy Performance Comparison', fontweight='bold', fontsize=14)
ax.set_ylabel('Portfolio Value ($)')
ax.set_xlabel('Date')
ax.legend()
ax.grid(True, alpha=0.3)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x:,.0f}'))

plt.show()

# Risk-Return scatter
fig, ax = plt.subplots(1, 1, figsize=(10, 8))

strategies = {
    'Raw Strategy': portfolio_returns,
    'Factor Neutralized': neutralized_data['Neutralized Returns'],
    'Vol Targeted': final_returns,
    'Benchmark': benchmark_returns
}

for name, returns in strategies.items():
    annual_return = returns.mean() * 252
    annual_vol = returns.std() * np.sqrt(252)
    ax.scatter(annual_vol, annual_return, s=100, label=name, alpha=0.8)
    ax.annotate(name, (annual_vol, annual_return), xytext=(5, 5), 
               textcoords='offset points', fontsize=10)

ax.set_xlabel('Annualized Volatility')
ax.set_ylabel('Annualized Return')
ax.set_title('Risk-Return Profile', fontweight='bold', fontsize=14)
ax.grid(True, alpha=0.3)
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.1%}'))
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.1%}'))

plt.show()

In [ ]:
# Step 6: Attribution Analysis using attribution.py
print("Performing attribution analysis...")

# Prepare data for attribution analysis
# Create factor returns for attribution
factor_returns = pd.DataFrame(index=returns_series.index)
factor_returns['Market'] = backtest_data['Market_Factor'] * 0.001  # Convert to return scale
factor_returns['Size'] = backtest_data['Size_Factor'] * 0.0005
factor_returns['Momentum'] = returns_series.rolling(20).mean()  # Momentum factor
factor_returns = factor_returns.fillna(0)

# Asset returns for attribution (individual asset contributions)
asset_returns = pd.DataFrame()
for ticker in tickers:
    return_col = f'{ticker}_return'
    if return_col in backtest_data.columns:
        asset_returns[ticker] = backtest_data[return_col]

asset_returns = asset_returns.fillna(0)

print(f"Running attribution analysis for {len(asset_returns.columns)} assets and {len(factor_returns.columns)} factors")

try:
    # Calculate factor exposures
    exposures = calculate_factor_exposures(asset_returns, factor_returns)
    print("✓ Factor exposures calculated")
    print("\nFactor Exposures:")
    print(exposures.round(3))
    
    # Calculate attributed PnL
    attributed_pnl = attribute_pnl(asset_returns, factor_returns, exposures)
    print("✓ PnL attribution calculated")
    
    # Generate attribution report
    attribution_report = generate_attribution_report(attributed_pnl)
    print("\n" + "="*50)
    print("ATTRIBUTION ANALYSIS REPORT")
    print("="*50)
    print(attribution_report)
    
except Exception as e:
    print(f"Attribution analysis failed: {e}")
    print("Creating simplified attribution analysis...")
    
    # Simplified attribution - contribution by asset
    portfolio_weights = portfolio_positions.abs().div(portfolio_positions.abs().sum(axis=1), axis=0)
    portfolio_weights = portfolio_weights.fillna(0)
    
    asset_contributions = {}
    for ticker in tickers:
        if ticker in asset_returns.columns and ticker in portfolio_weights.columns:
            contribution = (portfolio_weights[ticker] * asset_returns[ticker]).fillna(0)
            asset_contributions[ticker] = {
                'Total_Contribution': contribution.sum(),
                'Average_Weight': portfolio_weights[ticker].mean(),
                'Contribution_Volatility': contribution.std()
            }
    
    attribution_simple = pd.DataFrame(asset_contributions).T
    print("\nSimplified Asset Attribution:")
    print(attribution_simple.round(4))

# Attribution visualization
if 'attributed_pnl' in locals():
    print("Creating attribution visualizations...")
    
    # Plot attribution for top contributing assets
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Top 4 assets by total attributed PnL
    top_assets = attributed_pnl.sum().abs().nlargest(4).index
    
    for i, asset in enumerate(top_assets):
        ax = axes[i//2, i%2]
        cumulative_attribution = attributed_pnl[asset].cumsum()
        ax.plot(cumulative_attribution.index, cumulative_attribution, linewidth=2)
        ax.set_title(f'Cumulative Attribution - {asset}', fontweight='bold')
        ax.set_ylabel('Cumulative Attributed PnL')
        ax.grid(True, alpha=0.3)
        ax.axhline(y=0, color='gray', linestyle='-', alpha=0.5)
    
    plt.tight_layout()
    plt.show()

# Factor exposure analysis
if 'exposures' in locals():
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    
    # Heatmap of factor exposures
    sns.heatmap(exposures, annot=True, cmap='RdBu_r', center=0, 
               ax=ax, cbar_kws={'label': 'Factor Exposure'})
    ax.set_title('Asset Factor Exposures', fontweight='bold', fontsize=14)
    ax.set_xlabel('Assets')
    ax.set_ylabel('Factors')
    
    plt.show()
    
    # Factor contribution over time
    factor_contribution = pd.DataFrame(index=factor_returns.index)
    for factor in factor_returns.columns:
        total_exposure = exposures[factor].sum()
        factor_contribution[factor] = factor_returns[factor] * total_exposure
    
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    factor_contribution.cumsum().plot(ax=ax, linewidth=2)
    ax.set_title('Cumulative Factor Contributions', fontweight='bold', fontsize=14)
    ax.set_ylabel('Cumulative Contribution')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.show()

In [ ]:
# Step 7: Risk Analysis and Stress Testing
print("Performing risk analysis and stress testing...")

# Calculate comprehensive risk metrics
risk_metrics = {}

# Value at Risk (VaR) and Expected Shortfall (ES)
for confidence in [0.95, 0.99]:
    var = np.percentile(returns_series, (1-confidence)*100)
    es = returns_series[returns_series <= var].mean()
    risk_metrics[f'VaR_{int(confidence*100)}'] = var
    risk_metrics[f'ES_{int(confidence*100)}'] = es

# Tail risk metrics
risk_metrics['Skewness'] = returns_series.skew()
risk_metrics['Kurtosis'] = returns_series.kurtosis()
risk_metrics['Tail_Ratio'] = np.percentile(returns_series, 95) / abs(np.percentile(returns_series, 5))

# Maximum consecutive losses
consecutive_losses = 0
max_consecutive = 0
for ret in returns_series:
    if ret < 0:
        consecutive_losses += 1
        max_consecutive = max(max_consecutive, consecutive_losses)
    else:
        consecutive_losses = 0

risk_metrics['Max_Consecutive_Losses'] = max_consecutive

print("\n" + "="*50)
print("RISK ANALYSIS REPORT")
print("="*50)

for metric, value in risk_metrics.items():
    if 'VaR' in metric or 'ES' in metric:
        print(f"{metric}: {value:.4f} ({value:.2%})")
    elif metric in ['Skewness', 'Kurtosis', 'Tail_Ratio']:
        print(f"{metric}: {value:.3f}")
    else:
        print(f"{metric}: {value}")

# Stress testing scenarios
print("\n" + "="*50)
print("STRESS TESTING SCENARIOS")
print("="*50)

stress_scenarios = {
    'Market_Crash_2008': -0.20,    # -20% market shock
    'Flash_Crash': -0.10,          # -10% sudden drop
    'High_Volatility': 0.05,       # +5% with high vol
    'Liquidity_Crisis': -0.15      # -15% with liquidity issues
}

for scenario_name, shock in stress_scenarios.items():
    # Apply shock to portfolio
    stressed_return = returns_series.mean() + shock
    stressed_portfolio_value = initial_capital * (1 + stressed_return)
    loss_amount = initial_capital - stressed_portfolio_value
    loss_percentage = loss_amount / initial_capital
    
    print(f"{scenario_name}:")
    print(f"  Shock Applied: {shock:.1%}")
    print(f"  Portfolio Value: ${stressed_portfolio_value:,.0f}")
    print(f"  Loss Amount: ${loss_amount:,.0f}")
    print(f"  Loss Percentage: {loss_percentage:.2%}")
    print()

# Risk visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Returns distribution with VaR
ax1 = axes[0, 0]
ax1.hist(returns_series, bins=50, alpha=0.7, density=True, edgecolor='black')
ax1.axvline(x=risk_metrics['VaR_95'], color='red', linestyle='--', 
           label=f"VaR 95%: {risk_metrics['VaR_95']:.3f}")
ax1.axvline(x=risk_metrics['VaR_99'], color='darkred', linestyle='--', 
           label=f"VaR 99%: {risk_metrics['VaR_99']:.3f}")
ax1.set_title('Returns Distribution with VaR', fontweight='bold')
ax1.set_xlabel('Daily Returns')
ax1.set_ylabel('Density')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Rolling volatility
ax2 = axes[0, 1]
rolling_vol = returns_series.rolling(30).std() * np.sqrt(252)
ax2.plot(rolling_vol.index, rolling_vol, linewidth=2)
ax2.axhline(y=rolling_vol.mean(), color='red', linestyle='--', 
           label=f'Average: {rolling_vol.mean():.1%}')
ax2.set_title('30-Day Rolling Volatility (Annualized)', fontweight='bold')
ax2.set_ylabel('Volatility')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.1%}'))

# Q-Q plot for normality check
ax3 = axes[1, 0]
from scipy import stats
stats.probplot(returns_series.dropna(), dist="norm", plot=ax3)
ax3.set_title('Q-Q Plot (Normal Distribution)', fontweight='bold')
ax3.grid(True, alpha=0.3)

# Underwater plot (drawdown duration)
ax4 = axes[1, 1]
cumulative = (1 + returns_series).cumprod()
running_max = cumulative.expanding().max()
underwater = (cumulative - running_max) / running_max
ax4.fill_between(underwater.index, underwater, 0, alpha=0.7, color='red')
ax4.set_title('Underwater Plot (Drawdown Duration)', fontweight='bold')
ax4.set_ylabel('Drawdown')
ax4.grid(True, alpha=0.3)
ax4.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'{x:.1%}'))

plt.tight_layout()
plt.show()

In [ ]:
# Step 8: Position Analysis and Trade Analytics
print("Analyzing positions and trade characteristics...")

# Position analysis
position_analysis = {}

for ticker in tickers:
    if ticker in portfolio_positions.columns:
        positions = portfolio_positions[ticker]
        non_zero_positions = positions[positions != 0]
        
        if len(non_zero_positions) > 0:
            position_analysis[ticker] = {
                'Total_Trades': len(non_zero_positions),
                'Average_Position_Size': non_zero_positions.abs().mean(),
                'Max_Position_Size': non_zero_positions.abs().max(),
                'Long_Positions': (non_zero_positions > 0).sum(),
                'Short_Positions': (non_zero_positions < 0).sum(),
                'Position_Days': len(non_zero_positions),
                'Position_Frequency': len(non_zero_positions) / len(positions)
            }

position_df = pd.DataFrame(position_analysis).T
print("\nPosition Analysis by Asset:")
print(position_df.round(4))

# Trade duration analysis
print("\nTrade Duration Analysis:")
trade_durations = []
current_position = 0
trade_start = None

for date, positions_row in portfolio_positions.iterrows():
    total_position = positions_row.abs().sum()
    
    if total_position > 0 and current_position == 0:
        # New trade started
        trade_start = date
        current_position = total_position
    elif total_position == 0 and current_position > 0:
        # Trade ended
        if trade_start is not None:
            duration = (date - trade_start).days
            trade_durations.append(duration)
        current_position = 0

if trade_durations:
    print(f"Total Trades: {len(trade_durations)}")
    print(f"Average Trade Duration: {np.mean(trade_durations):.1f} days")
    print(f"Median Trade Duration: {np.median(trade_durations):.1f} days")
    print(f"Min Trade Duration: {min(trade_durations)} days")
    print(f"Max Trade Duration: {max(trade_durations)} days")

# Turnover analysis
daily_turnover = portfolio_positions.diff().abs().sum(axis=1)
average_turnover = daily_turnover.mean()
annual_turnover = average_turnover * 252

print(f"\nTurnover Analysis:")
print(f"Average Daily Turnover: {average_turnover:.4f}")
print(f"Estimated Annual Turnover: {annual_turnover:.2f}x")

# Transaction cost impact
transaction_costs = daily_turnover * commission  # Apply commission rate
net_returns_after_costs = returns_series - transaction_costs
cumulative_cost_impact = transaction_costs.cumsum()

cost_impact_metrics = {
    'Total_Transaction_Costs': transaction_costs.sum(),
    'Average_Daily_Costs': transaction_costs.mean(),
    'Cost_Impact_on_Returns': (returns_series.mean() - net_returns_after_costs.mean()) * 252,
    'Cost_as_Percent_of_Returns': (transaction_costs.sum() / returns_series.sum()) if returns_series.sum() != 0 else 0
}

print(f"\nTransaction Cost Analysis:")
for metric, value in cost_impact_metrics.items():
    if 'Percent' in metric:
        print(f"{metric}: {value:.2%}")
    elif 'Impact' in metric or 'Daily' in metric:
        print(f"{metric}: {value:.4f}")
    else:
        print(f"{metric}: {value:.6f}")

# Position and trade visualization
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Position concentration over time
ax1 = axes[0, 0]
position_concentration = portfolio_positions.abs().sum(axis=1)
ax1.plot(position_concentration.index, position_concentration, linewidth=2)
ax1.set_title('Total Position Concentration Over Time', fontweight='bold')
ax1.set_ylabel('Total Absolute Positions')
ax1.grid(True, alpha=0.3)

# Position distribution by asset
ax2 = axes[0, 1]
if len(position_df) > 0:
    position_df['Position_Days'].plot(kind='bar', ax=ax2, alpha=0.7)
    ax2.set_title('Position Days by Asset', fontweight='bold')
    ax2.set_ylabel('Number of Days with Positions')
    ax2.tick_params(axis='x', rotation=45)
    ax2.grid(True, alpha=0.3)

# Trade duration histogram
ax3 = axes[1, 0]
if trade_durations:
    ax3.hist(trade_durations, bins=20, alpha=0.7, edgecolor='black')
    ax3.axvline(x=np.mean(trade_durations), color='red', linestyle='--', 
               label=f'Mean: {np.mean(trade_durations):.1f} days')
    ax3.set_title('Trade Duration Distribution', fontweight='bold')
    ax3.set_xlabel('Duration (days)')
    ax3.set_ylabel('Frequency')
    ax3.legend()
    ax3.grid(True, alpha=0.3)

# Cumulative transaction costs
ax4 = axes[1, 1]
ax4.plot(cumulative_cost_impact.index, cumulative_cost_impact, 
         linewidth=2, color='red', label='Cumulative Costs')
ax4_twin = ax4.twinx()
ax4_twin.plot(returns_series.cumsum().index, returns_series.cumsum(), 
              linewidth=2, color='blue', alpha=0.7, label='Cumulative Returns')
ax4.set_title('Transaction Costs vs Returns', fontweight='bold')
ax4.set_ylabel('Cumulative Transaction Costs', color='red')
ax4_twin.set_ylabel('Cumulative Returns', color='blue')
ax4.legend(loc='upper left')
ax4_twin.legend(loc='upper right')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Step 9: Generate Comprehensive Report and Save Results
print("Generating comprehensive performance report...")

# Create final performance summary
final_report = {
    'Strategy_Overview': {
        'Strategy_Type': strategy_config['type'],
        'Backtest_Period': f"{backtest_start} to {backtest_end}",
        'Number_of_Assets': len(tickers),
        'Initial_Capital': initial_capital,
        'Final_Portfolio_Value': portfolio_backtest['Portfolio Value'].iloc[-1],
        'Total_Return': performance_metrics['Total Return'],
        'Annualized_Return': performance_metrics['Annualized Return'],
        'Max_Drawdown': performance_metrics['Max Drawdown']
    },
    'Risk_Metrics': {
        'Sharpe_Ratio': sharpe_ratio,
        'Sortino_Ratio': sortino_ratio,
        'Calmar_Ratio': calmar_ratio,
        'VaR_95': risk_metrics['VaR_95'],
        'Expected_Shortfall_95': risk_metrics['ES_95'],
        'Skewness': risk_metrics['Skewness'],
        'Kurtosis': risk_metrics['Kurtosis']
    },
    'Trading_Statistics': {
        'Win_Rate': win_rate,
        'Average_Daily_Return': returns_series.mean(),
        'Return_Volatility': returns_series.std(),
        'Best_Day': returns_series.max(),
        'Worst_Day': returns_series.min(),
        'Positive_Days': (returns_series > 0).sum(),
        'Negative_Days': (returns_series < 0).sum(),
        'Annual_Turnover': annual_turnover
    },
    'Cost_Analysis': cost_impact_metrics
}

# Display final report
print("\n" + "="*80)
print("STATISTICAL ARBITRAGE STRATEGY - FINAL PERFORMANCE REPORT")
print("="*80)

for section, metrics in final_report.items():
    print(f"\n{section.replace('_', ' ').upper()}:")
    print("-" * 50)
    
    for metric, value in metrics.items():
        metric_name = metric.replace('_', ' ')
        if isinstance(value, float):
            if any(keyword in metric.lower() for keyword in ['return', 'ratio', 'rate', 'impact']):
                if abs(value) < 0.001:
                    print(f"{metric_name}: {value:.6f}")
                elif abs(value) < 1:
                    print(f"{metric_name}: {value:.4f}")
                else:
                    print(f"{metric_name}: {value:.2f}")
            elif 'drawdown' in metric.lower() or 'var' in metric.lower():
                print(f"{metric_name}: {value:.4f} ({value:.2%})")
            elif 'value' in metric.lower() or 'capital' in metric.lower():
                print(f"{metric_name}: ${value:,.2f}")
            else:
                print(f"{metric_name}: {value:.4f}")
        else:
            print(f"{metric_name}: {value}")

# Save results to files
print(f"\n{'='*50}")
print("SAVING RESULTS")
print("="*50)

# Create results directory if it doesn't exist
results_dir = "../results"
os.makedirs(results_dir, exist_ok=True)

# Save key datasets
datasets_to_save = {
    'portfolio_returns.csv': returns_series,
    'portfolio_positions.csv': portfolio_positions,
    'signals.csv': signals,
    'backtest_results.csv': portfolio_backtest,
    'performance_metrics.csv': pd.Series(performance_metrics),
    'risk_metrics.csv': pd.Series(risk_metrics)
}

for filename, data in datasets_to_save.items():
    filepath = os.path.join(results_dir, filename)
    if isinstance(data, pd.DataFrame):
        data.to_csv(filepath)
    elif isinstance(data, pd.Series):
        data.to_csv(filepath, header=True)
    print(f"✓ Saved {filename}")

# Save comprehensive report as YAML
report_filepath = os.path.join(results_dir, 'final_performance_report.yaml')
with open(report_filepath, 'w') as f:
    yaml.dump(final_report, f, default_flow_style=False)
print(f"✓ Saved final_performance_report.yaml")

# Save strategy configuration
strategy_filepath = os.path.join(results_dir, 'strategy_config.yaml')
with open(strategy_filepath, 'w') as f:
    yaml.dump(config, f, default_flow_style=False)
print(f"✓ Saved strategy_config.yaml")

print(f"\nAll results saved to: {results_dir}")

# Generate executive summary
print(f"\n{'='*80}")
print("EXECUTIVE SUMMARY")
print("="*80)

total_return_pct = performance_metrics['Total Return']
annual_return_pct = performance_metrics['Annualized Return']
max_dd_pct = performance_metrics['Max Drawdown']

print(f"""
📊 STRATEGY PERFORMANCE OVERVIEW:
   • Strategy delivered {total_return_pct:.1%} total return over the backtest period
   • Annualized return of {annual_return_pct:.1%} with maximum drawdown of {max_dd_pct:.1%}
   • Sharpe ratio of {sharpe_ratio:.2f} indicates {('strong' if sharpe_ratio > 1 else 'moderate' if sharpe_ratio > 0.5 else 'weak')} risk-adjusted performance

⚡ SIGNAL EFFECTIVENESS:
   • Generated {signal_summary['Long_signals'].sum() + signal_summary['Short_signals'].sum()} total signals
   • Win rate of {win_rate:.1%} with average winning day of {positive_returns.mean():.2%}
   • Signal frequency averaged {signal_summary['Signal_frequency'].mean():.1%} across all assets

💰 COST IMPACT:
   • Transaction costs reduced returns by {cost_impact_metrics['Cost_Impact_on_Returns']:.2%} annually
   • Annual turnover of {annual_turnover:.1f}x indicates {('high' if annual_turnover > 3 else 'moderate' if annual_turnover > 1 else 'low')} trading frequency

🎯 RISK MANAGEMENT:
   • 95% VaR of {risk_metrics['VaR_95']:.2%} indicates daily loss threshold
   • {"Positive" if risk_metrics['Skewness'] > 0 else "Negative"} skew of {risk_metrics['Skewness']:.2f} shows return distribution characteristics
   • Maximum consecutive losses: {risk_metrics['Max_Consecutive_Losses']} days

✅ RECOMMENDATION: 
   Strategy shows {"promising" if sharpe_ratio > 1 and max_dd_pct > -0.2 else "mixed"} results with room for optimization in 
   {"signal generation" if win_rate < 0.55 else "cost management" if annual_turnover > 3 else "risk management"}.
""")

print(f"\n{'='*80}")
print("BACKTEST ANALYSIS COMPLETE!")
print("="*80)